In [9]:
from music21 import *
from comp2 import *

In [10]:
melody_in: stream.Stream = parse_midi("example_inputs/eyeofthetiger.mid")

print("Original melody:")
melody_in.show('midi');

Original melody:


In [11]:
print("Simple triads:")
simple_triads = get_triads(melody_in)

Simple triads:
['iii', 'ii5#2', 'ii5#2', 'ii5#2', 'iii', 'iii', 'ii5#2', 'iii', 'iii', 'ii5#2', 'ii5#2', 'ii5#2', 'iii', 'iii', 'ii5#2', 'iii', 'vi+64', 'I', 'vi+64', 'iii', 'vi+64', 'I', 'vi+64', 'vi6', 'ii5#2']


In [12]:
out_score = stream.Score()
out_score.insert(0, melody_in)
out_score.insert(0, simple_triads)

out_score.show('midi')